I have chosen the ResNet model. Because the ResNet model can train extremely deep networks efficiently and without experiencing the vanishing gradient problem. It can acquire intricate and abstract representations of the input data thanks to the usage of residual connections, which enhances performance on picture classification tasks. Its architecture is incredibly versatile and adaptable to our purpose. Cross-entropy loss is a popular and the best option for this situation in the context of image classification since it is an appropriate loss function for multi-class classification problems. Cross-entropy loss has the mathematical qualities that make it simple to compute gradients and guarantee that the gradient will always be positive. which is the best for this case.

In [906]:
import os
import torch
import torchvision
import tarfile
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [907]:
path = 'symbols_dataset' #or the path to the folder with the data set for question 3 without the redundant data (.Mac and the other one)
t_tfms = tt.Compose([tt.Grayscale(num_output_channels=1), tt.RandomHorizontalFlip(), tt.ToTensor()])
dataset = ImageFolder(path, transform=t_tfms)

tr_size = int(0.9 * len(dataset))
t_size = len(dataset) - tr_size
tr_data, t_data = random_split(dataset, [tr_size, t_size])
t_dl = DataLoader(tr_data, batch_size=100, shuffle=True, num_workers=2, pin_memory=True)
v_dl = DataLoader(t_data, batch_size=100, shuffle=True, num_workers=2, pin_memory=True)

In [908]:
def get_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
        
def swapDevice(data, device):
    if isinstance(data, (list,tuple)):
        return [swapDevice(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [909]:
class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield swapDevice(b, self.device)

    def __len__(self):
        return len(self.dl)

In [910]:
t_dl = DeviceDataLoader(t_dl, get_device())
v_dl = DeviceDataLoader(v_dl, get_device())

In [911]:
def accuracy(ops, lab):
    _, preds = torch.max(ops, dim=1)
    output = torch.tensor(torch.sum(preds == lab).item() / len(preds))
    return output

In [912]:
class ImageClassificationBase(nn.Module):
    def training_step(self, b):
        imgs, labels = b
        out = self(imgs)                  
        loss = F.cross_entropy(out, labels) 
        return loss
    
    def validation_step(self, b):
        imgs, labels = b 
        out = self(imgs)                    
        loss = F.cross_entropy(out, labels)   
        acc = accuracy(out, labels)           
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, ops):
        b_l = [x['val_loss'] for x in ops]
        e_loss = torch.stack(b_l).mean()   
        b_accs = [x['val_acc'] for x in ops]
        e_acc = torch.stack(b_accs).mean()      
        return {'val_loss': e_loss.item(), 'val_acc': e_acc.item()}
    
    def epoch_end(self, e, result):
        print("E# [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(e, result['train_loss'], result['val_loss'], result['val_acc']))

In [913]:
def conv(in_c, out_c, pool=False):
    l = [nn.Conv2d(in_c, out_c, kernel_size=3, padding=1), nn.BatchNorm2d(out_c), nn.ReLU(inplace=True)]
    if pool: l.append(nn.MaxPool2d(2))
    return nn.Sequential(*l)

In [914]:
class ResNet9(ImageClassificationBase):           
    def __init__(self, in_channels, num_classes):
        super().__init__()
          
        self.conv1 = conv(in_channels, 96, pool=True)
        self.conv2 = conv(96, 192, pool=True)
        self.res1 = nn.Sequential(conv(192, 192), conv(192, 192))
        self.conv4 = conv(192, 384, pool=True)
        self.res2 = nn.Sequential(conv(384, 384), conv(384, 384))
        self.classifier = nn.Sequential(nn.MaxPool2d(4), nn.Flatten(), nn.Linear(384, num_classes))
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [915]:
model = swapDevice(ResNet9(1, 5), get_device())

In [916]:
@torch.no_grad()
def evaluate(m, val_l):
    m.eval()
    outputs = [m.validation_step(batch) for batch in val_l]
    return m.validation_epoch_end(outputs)


def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [917]:
def foc(n_of_es, lr, m, train_l, val_l, w_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    h = []

    opt = opt_func(m.parameters(), lr, weight_decay=w_decay)
    sc = torch.optim.lr_scheduler.OneCycleLR(opt, lr, epochs=n_of_es,  steps_per_epoch=len(train_l))
    
    for epoch in range(n_of_es):
        m.train()
        ls = []
        lrs = []
        for batch in train_l:
            loss = m.training_step(batch)
            ls.append(loss)
            loss.backward()
            if grad_clip: 
                nn.utils.clip_grad_value_(m.parameters(), grad_clip)    
            opt.step()
            opt.zero_grad()
            lrs.append(get_lr(opt))
            sc.step()

        r = evaluate(m, val_l)
        r['train_loss'] = torch.stack(ls).mean().item()
        r['lrs'] = lrs
        m.epoch_end(epoch, r)
        h.append(r)
    return h

In [918]:
opt_func = torch.optim.Adam
weight_decay = 1e-3
n_o_epochs = 5
m_lr = 1.1e-6
grad_clip = 0.03

In [919]:
h = [evaluate(model, v_dl)]
h += foc(n_o_epochs, m_lr, model, t_dl, v_dl, grad_clip=grad_clip, w_decay=weight_decay, opt_func=opt_func)

E# [0], train_loss: 2.6246, val_loss: 1.9487, val_acc: 0.3936
E# [1], train_loss: 1.1289, val_loss: 0.5547, val_acc: 0.8603
E# [2], train_loss: 0.4310, val_loss: 0.3390, val_acc: 0.9372
E# [3], train_loss: 0.3053, val_loss: 0.2792, val_acc: 0.9455
E# [4], train_loss: 0.2773, val_loss: 0.2670, val_acc: 0.9522


In [921]:
torch.save(model.state_dict(), 'weights.pkl')